For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

## Part 1
1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, <https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M>, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

3. To create the above dataframe:
 * The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
 * Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**.
 * More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that **M5A** is listed twice and has two neighborhoods: **Harbourfront** and **Regent Park**. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in **row 11** in the above table. (***Note:*** This problem seem to have been already corrected in the Wiki data.)
 * If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the same as the borough. (***Note:*** This problem seems to have been already corrected in the Wiki data.)
 * Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
 * In the last cell of your notebook, use the **.shape** method to print the number of rows of your dataframe.
 
4. Submit a link to your Notebook on your Github repository. (**10 marks**)

**Note**: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use *pandas* to read the table into a *pandas* dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: <http://beautiful-soup-4.readthedocs.io/en/latest/>

Use *pandas*, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

#### Import packages

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#### Scrape the list of Toronto's postal codes

In [2]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text
soup = BeautifulSoup(source, 'xml')
table = soup.find('table')

#### Convert table to df with postal code, borough, and neighbourhood info

In [3]:
def TableRowsToDataRows(row):
    '''Convert soup table row to list of data'''
    # Strip html and whitespace from each cell of row.
    return list(map(lambda cell: cell.get_text().strip(), row.find_all()))

# Get list of all rows with formated data and put into dataframe. First row of data is column names.
data = list(map(TableRowsToDataRows, table.find_all('tr')))
df = pd.DataFrame(data[1:], columns = data[0])
df.head(12)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


#### Clean dataframe
Remove rows where Borough is 'Not assigned'. 
There are no postal codes were Neighbourhood is subsequently not assigned. Postal Codes with multiple neighbourhoods have already been grouped. 

In [4]:
df=df[df['Borough']!='Not assigned']
df[df['Neighbourhood']=='Not assigned']

,Postal Code,Borough,Neighbourhood


#### Review dataframe
Reset index and view dataframe. Examine it's shape.

In [5]:
df.reset_index(drop=True, inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
df.shape

(103, 3)

## Part 2
Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

##### Note:
In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. 

#### ***Note:***
Using `geocode` with Google's API is no longer free. Based on the suggestions from others, I've instead implemented this using `arcgis` instead of `google`. 

In [7]:
# Run this code if geocoder is not installed.
#!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

Define a function that uses `geocode` to fetch latitude and longitude for the postal code from a row of the dataframe.

In [8]:
def get_geocode_for_row(row):
    '''
    Fetches coordinates for a postal code given a row in the dataframe.
    Returns a list of the form [latititude, longitude].
    '''
    print('\rFetching data for postal code: ' + row['Postal Code'], end='')
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(row['Postal Code']))
    return [round(coord,5) for coord in g.geojson['features'][0]['geometry']['coordinates']][::-1]

Apply this to each row of the dataframe and add columns for 'Latitude' and 'Longitude'. ***Worning:*** This takes a long time for some reason. I've written in the code a way to track the progress by displaying the postal code that is currently being checked.

In [9]:
df['Latitude'], df['Longitude'] = zip(*df.apply(get_geocode_for_row, axis=1))
print('\rDone!',end='                              ')

Done!                              

In [10]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75188,-79.33036
1,M4A,North York,Victoria Village,43.73042,-79.31282
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72321,-79.45141
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302


##### Alternate method
An alternate method (since the geocoder method may be unreliable) is to simply fetch the geospatial data that has already been fetched by the course coordinaters. I've commented this method out here.

***Note:*** The geospatial coordinates from `arcgis` are slightly different from those provided by Google. 

In [12]:
#geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
#geo_df.rename(columns={'Postalcode':'Postal Code'}, inplace=True)
#geo_df.head()
#df_merge = pd.merge(df, geo_df, on='Postal Code')
#df_merge.head()